In [1]:
import zipfile
import os
import pathlib
import random

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

[Link to file download(maybe need to login on kaggle)](https://www.kaggle.com/datasets/ravirajsinh45/real-life-industrial-dataset-of-casting-product)

In [4]:
zip_file_extraction = zipfile.ZipFile("casting-product-quality-dataset-kaggle.zip")
zip_file_extraction.extractall()
zip_file_extraction.close()

In [6]:
!ls casting_512x512/casting_512x512

def_front  ok_front


In [9]:
for dirpath, dirnames, filenames in os.walk("casting_data"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 1 directories and 0 images in 'casting_data'.
There are 2 directories and 0 images in 'casting_data/casting_data'.
There are 2 directories and 0 images in 'casting_data/casting_data/train'.
There are 0 directories and 3758 images in 'casting_data/casting_data/train/def_front'.
There are 0 directories and 2875 images in 'casting_data/casting_data/train/ok_front'.
There are 2 directories and 0 images in 'casting_data/casting_data/test'.
There are 0 directories and 453 images in 'casting_data/casting_data/test/def_front'.
There are 0 directories and 262 images in 'casting_data/casting_data/test/ok_front'.


In [10]:
# Set the seed
tf.random.set_seed(42)

# Preprocess data (get all of the pixel values between 1 and 0, also called scaling/normalization)
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

# Setup the train and test directories
train_dir = "casting_data/casting_data/train/"
test_dir = "casting_data/casting_data/test/"

# Import data from directories and turn it into batches
train_data = train_datagen.flow_from_directory(train_dir,
                                               batch_size=32, # number of images to process at a time 
                                               target_size=(224, 224), # convert all images to be 224 x 224
                                               class_mode="binary", # type of problem we're working on
                                               seed=42)

valid_data = valid_datagen.flow_from_directory(test_dir,
                                               batch_size=32,
                                               target_size=(224, 224),
                                               class_mode="binary",
                                               seed=42)

# Create a CNN model (same as Tiny VGG - https://poloclub.github.io/cnn-explainer/)
model_1 = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(filters=10, 
                         kernel_size=3, # can also be (3, 3)
                         activation="relu", 
                         input_shape=(224, 224, 3)), # first layer specifies input shape (height, width, colour channels)
  tf.keras.layers.Conv2D(10, 3, activation="relu"),
  tf.keras.layers.MaxPool2D(pool_size=2, # pool_size can also be (2, 2)
                            padding="valid"), # padding can also be 'same'
  tf.keras.layers.Conv2D(10, 3, activation="relu"),
  tf.keras.layers.Conv2D(10, 3, activation="relu"), # activation='relu' == tf.keras.layers.Activations(tf.nn.relu)
  tf.keras.layers.MaxPool2D(2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(1, activation="sigmoid") # binary activation output
])

# Compile the model
model_1.compile(loss="binary_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

# Fit the model
history_1 = model_1.fit(train_data,
                        epochs=5,
                        steps_per_epoch=len(train_data),
                        validation_data=valid_data,
                        validation_steps=len(valid_data))

Found 6633 images belonging to 2 classes.
Found 715 images belonging to 2 classes.
Epoch 1/5
208/208 [==============================] - 31s 91ms/step - loss: 0.5113 - accuracy: 0.7460 - val_loss: 0.3168 - val_accuracy: 0.8783
Epoch 2/5
208/208 [==============================] - 19s 91ms/step - loss: 0.2577 - accuracy: 0.8991 - val_loss: 0.2089 - val_accuracy: 0.9147
Epoch 3/5
208/208 [==============================] - 18s 89ms/step - loss: 0.1568 - accuracy: 0.9447 - val_loss: 0.0973 - val_accuracy: 0.9720
Epoch 4/5
208/208 [==============================] - 19s 89ms/step - loss: 0.0989 - accuracy: 0.9705 - val_loss: 0.2318 - val_accuracy: 0.8825
Epoch 5/5
208/208 [==============================] - 19s 90ms/step - loss: 0.0815 - accuracy: 0.9750 - val_loss: 0.1234 - val_accuracy: 0.9552


In [12]:
model_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 10)      280       
                                                                 
 conv2d_1 (Conv2D)           (None, 220, 220, 10)      910       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 110, 110, 10)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 108, 108, 10)      910       
                                                                 
 conv2d_3 (Conv2D)           (None, 106, 106, 10)      910       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 53, 53, 10)       0         
 2D)                                                    

In [17]:
# Set the seed
tf.random.set_seed(42)

# Preprocess data (get all of the pixel values between 1 and 0, also called scaling/normalization)
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

# Setup the train and test directories
train_dir = "casting_data/casting_data/train/"
test_dir = "casting_data/casting_data/test/"

# Import data from directories and turn it into batches
train_data = train_datagen.flow_from_directory(train_dir,
                                               batch_size=16, # number of images to process at a time 
                                               target_size=(224, 224), # convert all images to be 224 x 224
                                               class_mode="binary", # type of problem we're working on
                                               seed=42)

valid_data = valid_datagen.flow_from_directory(test_dir,
                                               batch_size=16,
                                               target_size=(224, 224),
                                               class_mode="binary",
                                               seed=42)

# Create a CNN model (same as Tiny VGG - https://poloclub.github.io/cnn-explainer/)
model_2 = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(filters=15, 
                         kernel_size=3, # can also be (3, 3)
                         activation="relu", 
                         input_shape=(224, 224, 3)), # first layer specifies input shape (height, width, colour channels)
  tf.keras.layers.Conv2D(10, 3, activation="relu"),
  tf.keras.layers.MaxPool2D(pool_size=2, # pool_size can also be (2, 2)
                            padding="valid"), # padding can also be 'same'
  tf.keras.layers.Conv2D(10, 3, activation="relu"),
  tf.keras.layers.Conv2D(10, 3, activation="relu"),
  tf.keras.layers.Conv2D(10, 3, activation="relu"), # activation='relu' == tf.keras.layers.Activations(tf.nn.relu)
  tf.keras.layers.MaxPool2D(2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(1, activation="sigmoid") # binary activation output
])

# Compile the model
model_2.compile(loss="binary_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

# Fit the model
history_2 = model_2.fit(train_data,
                        epochs=5,
                        steps_per_epoch=len(train_data),
                        validation_data=valid_data,
                        validation_steps=len(valid_data))

Found 6633 images belonging to 2 classes.
Found 715 images belonging to 2 classes.
Epoch 1/5
415/415 [==============================] - 20s 46ms/step - loss: 0.5118 - accuracy: 0.7338 - val_loss: 0.3021 - val_accuracy: 0.8979
Epoch 2/5
415/415 [==============================] - 19s 46ms/step - loss: 0.1957 - accuracy: 0.9278 - val_loss: 0.1549 - val_accuracy: 0.9357
Epoch 3/5
415/415 [==============================] - 19s 45ms/step - loss: 0.1142 - accuracy: 0.9614 - val_loss: 0.0923 - val_accuracy: 0.9692
Epoch 4/5
415/415 [==============================] - 19s 46ms/step - loss: 0.0832 - accuracy: 0.9739 - val_loss: 0.0480 - val_accuracy: 0.9846
Epoch 5/5
415/415 [==============================] - 19s 45ms/step - loss: 0.0621 - accuracy: 0.9815 - val_loss: 0.0523 - val_accuracy: 0.9776


In [15]:
model_2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 222, 222, 15)      420       
                                                                 
 conv2d_5 (Conv2D)           (None, 220, 220, 10)      1360      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 110, 110, 10)     0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 108, 108, 10)      910       
                                                                 
 conv2d_7 (Conv2D)           (None, 106, 106, 10)      910       
                                                                 
 conv2d_8 (Conv2D)           (None, 104, 104, 10)      910       
                                                      